In [6]:
# nuclio: ignore
import nuclio

In [7]:
import os
import json
import importlib
from cloudpickle import load

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import label_binarize
from sklearn.utils.multiclass import unique_labels

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem
from mlrun.artifacts import TableArtifact, PlotArtifact

from mlutils.models import get_model_configs
from mlutils.plots import plot_roc, plot_importance, gcf_clear

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

def _gcf_clear(plt):
    plt.cla()
    plt.clf()
    plt.close()        

def test_classifier(
    context: MLClientCtx,
    models_dir: str, 
    test_set: str,
    label_column: str,
    score_method: str = 'micro',
    key: str = "",
    plots_dest: str = "plots"
) -> None:
    """Test one or more classifier models against held-out dataset
    
    Using held-out test features, evaluates the peformance of the estimated model
    
    Can be part of a kubeflow pipeline as a test step that is run post EDA and 
    training/validation cycles
    
    :param context:         the function context
    :param models_dir:      artifact models representing a folder or a folder
    :param test_set:        test features and labels
    :param label_column:    column name for ground truth labels
    :param score_method:    for multiclass classification
    :param key:             key for results artifact (maybe just a dir of artifacts for test like plots_dest)
    :param plots_dest:       dir for test plots
    """
    xtest = pd.read_parquet(str(test_set))
    ytest = xtest.pop(label_column)
    
    context.header = list(xtest.columns.values)
    
    def _eval_model(model):
        # enclose all except model
        ytestb = label_binarize(ytest, classes=ytest.unique())
        clf = load(open(os.path.join(str(models_dir), "model")+".pkl", "rb"))
        if callable(getattr(clf, "predict_proba")):
            y_score = clf.predict_proba(xtest.values)
            ypred = clf.predict(xtest.values)
            context.logger.info(f"y_score.shape {y_score.shape}")
            context.logger.info(f"ytestb.shape {ytestb.shape}")
            plot_roc(context, ytestb, y_score, key=f"roc", plots_dir=plots_dest)
        else:
            ypred = clf.predict(xtest.values) # refactor
            y_score = None
            
        gcf_clear(plt)
        # use sklearn >= v0.22 built in:
       
        metrics.plot_confusion_matrix(clf, xtest, ytest, 
                                      labels=ytest.unique(), normalize='true') 
        
        context.log_artifact(PlotArtifact("confusion", body=plt.gcf()), 
                             local_path=f"{plots_dest}/confusion.html")        
    
        if hasattr(clf, "feature_importances_"):
            plot_importance(context, clf, key=f"featimp")
        average_precision = metrics.average_precision_score(ytestb[:,:-1], y_score, average=score_method)
        context.log_result(f"accuracy", float(clf.score(xtest.values, ytest.values)))
        context.log_result(f"rocauc", metrics.roc_auc_score(ytestb, y_score))
        context.log_result(f"f1_score", metrics.f1_score(ytest.values, ypred, average=score_method))
        context.log_result(f"avg_precscore", average_precision)
    
    best_model = None
    for model in os.listdir(str(models_dir)):
        if model.endswith('.pkl'):
            _eval_model(model)
            # HACK: there is only one model here
            best_model = model

    # log 'best model' as artifact
    context.log_artifact('TODAYS-MODELS-TEST-REPORT', local_path=best_model)
    context.log_artifact('DEPLOY', body=b'true', local_path='DEPLOY')


In [8]:
# nuclio: end-code

### save

In [ ]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("test_classifier", kind="job", with_doc=True,
                      handler=test_classifier, image="mlrun/ml-models")

# add metadata (for templates and reuse)
fn.spec.default_handler = "test_classifier"
fn.spec.description = "test a classifier using held-out or new data"
fn.metadata.categories = ["models", "testing"]
fn.spec.image_pull_policy = "Always"
fn.metadata.labels = {"author": "yjb"}

fn.save()
fn.export("function.yaml")

### test

In [ ]:
from mlrun import import_function, mount_v3io

func = import_function("hub://test_classifier").apply(mount_v3io())
# func = import_function("function.yaml").apply(mlrun.mount_v3io())

task_params = {
    "name" : "tasks - test classifier",
    "params": {
        # Ina pipeline setting, the models_dir parameter would be the output of a training step
        "models_dir"    : "/User/artifacts/models",
        "test_set"      : "/User/artifacts/test_set.parquet",
        "label_column"  : "labels"}}

from mlrun import NewTask
run = func.run(NewTask(**task_params), artifact_path="/User/artifacts")